# pg_server_open_transactions 
Description: Shows the top X open transactions in the server. 
Version: 0.1

## Configuration
Open a connection to the DB. Use a Connection String stored in a .cfg file

In [2]:
import sqlalchemy
import pandas as pd
import configparser
import matplotlib.pyplot as plt 

# Read the PG connection string from the Config file
config = configparser.ConfigParser() 
config.read_file(open(r'../ipynb.cfg'))
con_str = config.get('con_str', 'PG_PLATFORM') 
engine = sqlalchemy.create_engine(con_str)
# print("Connecting with engine " + str(engine))
try:
    connection = engine.connect()
except (Exception, sqlalchemy.exc.SQLAlchemyError) as error:
    print("Error while connecting to PostgreSQL database:", error)



## Idle Connections with an open transaction 


In [3]:

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

sql_command = """
select * 
from pg_stat_activity
where (state = 'idle in transaction')
    and xact_start is not null;
"""


try:
    # Execute the SQL command
   
    df = pd.read_sql_query(sql_command, connection)
    print(df)
except (SQLAlchemyError, ValueError) as e:
    # Handle any errors or raised exceptions
    raise e


Empty DataFrame
Columns: [datid, datname, pid, leader_pid, usesysid, usename, application_name, client_addr, client_hostname, client_port, backend_start, xact_start, query_start, state_change, wait_event_type, wait_event, state, backend_xid, backend_xmin, query_id, query, backend_type]
Index: []

[0 rows x 22 columns]


## Open Transactions
Shows transactions open for a long time. The default is ```interval '5 minutes'```. Feel free to change it.

In [4]:
query_open_transactions = """
SELECT 
		pg_stat_activity.application_name,
    pg_stat_activity.state,
    pg_stat_activity.wait_event_type,
    pg_stat_activity.wait_event,
    pg_stat_activity.query,
    pg_stat_activity.pid,
    pg_stat_activity.client_addr,
    age(now(), pg_stat_activity.query_start) AS running_time
FROM pg_stat_activity
WHERE (pg_stat_activity.state = 'active'::text) 
	AND pg_stat_activity.pid <> pg_backend_pid()
  AND (now() - pg_stat_activity.query_start) > interval '5 minutes'
ORDER BY running_time DESC; 
"""

df = pd.read_sql_query(query_open_transactions, connection)
df

,application_name,state,wait_event_type,wait_event,query,pid,client_addr,running_time


## Kill a Transaction
The function ```pg_terminate_backend``` terminates the session whose backend process has the specified process ID. Reousrce: https://www.postgresql.org/docs/15/functions-admin.html

In [ ]:
## Kill a single PID - forceful
"SELECT pg_terminate_backend(<PID>);"
## Kill a single PID - graceful
"SELECT pg_cancel_backend(<PID>);"

## Kill all idle connection
"""
FROM pg_stat_activity 
WHERE datname='db'
  AND state = 'idle in transaction'
"""